In [1]:
import pyspark #importing pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum #importing sum and count

In [2]:
spark = SparkSession.builder.appName("handson").getOrCreate() #creating spark session handson

In [3]:
df = spark.read.csv("/home/jovyan/titanic.csv", header = True) #reading CSV
df.show(5) #showing 5 rows from CSV
df.printSchema() # print Schema of DF

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [4]:
df.createOrReplaceTempView("titanic")  # create 

# A.How many people survive and how many people dead

In [5]:
survived_count = df.groupBy('Survived').agg(count('*').alias('count')) # grouped by Survivved and counting the rows
print("People Survived : \n ")
survived_count.filter(survived_count["Survived"] == 1).show() # 1 is survived -> filtering only survived count
print("People Dead : \n ")
survived_count.filter(survived_count["Survived"] == 0).show() # 0 is dead -> filtering only death count 

People Survived : 
 
+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
+--------+-----+

People Dead : 
 
+--------+-----+
|Survived|count|
+--------+-----+
|       0|  549|
+--------+-----+



In [6]:
print("People Survived : \n ")
spark.sql(
    "SELECT COUNT(*) AS Count_alive FROM titanic WHERE Survived=1"
).show() # Selecting alive people from titanic csv

print("People Death : \n ")
spark.sql(
    "select COUNT(*) AS Count_dead FROM titanic WHERE Survived=0"
).show() # Selecting dead people from titanic csv


People Survived : 
 
+-----------+
|Count_alive|
+-----------+
|        342|
+-----------+

People Death : 
 
+----------+
|Count_dead|
+----------+
|       549|
+----------+



# doing calculation for people with no siblings and death count using group by

In [7]:
no_siblings_df = df.filter(df['SibSp'] == 0) #filtering people who does not have siblings
no_siblings_df.show(5) #showing only 5 rows from dataframe
no_sib_count = no_siblings_df.groupBy('Survived').agg(count('*').alias('count')) #counting how many people died and survived among not having siblings
no_sib_count.show() 

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583| null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|  54|    0|    0|           17463|51.8625|  E46|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|  27|    0|    2|          347742|11.1333| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

# B.How many people survive how does not have sibling

In [8]:
no_sib_count.filter(no_sib_count['Survived'] == 1).show() #showing only alive people count without sibiling from above computation

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  210|
+--------+-----+



In [9]:
spark.sql(
    "select COUNT(*) AS Count_alive FROM titanic where Survived= 1 and SibSp = 0"
).show() # counting rows where survived is 1 and sibilings is 0

+-----------+
|Count_alive|
+-----------+
|        210|
+-----------+



# C.How many people died Who does not have sibling

In [10]:
no_sib_count.filter(no_sib_count['Survived'] == 0).show()  #showing only dead people count without sibilings from above computation

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  398|
+--------+-----+



In [11]:
spark.sql(
    "select COUNT(*) AS Count_alive FROM titanic where Survived= 0 and SibSp = 0"
).show() # counting rows where survived is 0 (dead) and sibilings is 0

+-----------+
|Count_alive|
+-----------+
|        398|
+-----------+



# D.Remove the NAN data from cabin and display updated list

In [12]:
not_null_cabin = df.filter(df["Cabin"].isNotNull()) # find not null in cabin
not_null_cabin.show(5) # showing 5 rows

+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|   17463|51.8625|  E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|  4|    1|    1| PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female| 58|    0|    0|  113783|  26.55| C103|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
only showing top 5 rows



In [13]:
spark.sql("SELECT * FROM titanic WHERE Cabin IS NOT NULL").show(5)# selecting rows with cabin as not null and showing 5 rows

+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|   17463|51.8625|  E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|  4|    1|    1| PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female| 58|    0|    0|  113783|  26.55| C103|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+--------+-------+-----+--------+
only showing top 5 rows



# E.How many male who died and how many female died?

In [14]:
grouped_survived = df.filter(df['Survived'] == 0) #filtering dataframe with survivedd column as 0 == dead people
males_females_died_count = grouped_survived.groupBy('Sex').agg(count('*').alias('count'))#counting the sex by grouped by
print("Male and Female who died")
males_females_died_count.show()#showing the count from above computation

Male and Female who died
+------+-----+
|   Sex|count|
+------+-----+
|female|   81|
|  male|  468|
+------+-----+



In [15]:
spark.sql(
'''select Sex, COUNT(*) AS Count_alive FROM (
    select * from titanic where Survived = 0
) group by Sex
        
    '''
).show() #selecting sex column and counting the rows only from the subquery... which gives rows which has dead people 

+------+-----------+
|   Sex|Count_alive|
+------+-----------+
|female|         81|
|  male|        468|
+------+-----------+

